In [69]:
import requests
from bs4 import BeautifulSoup as bs
import time, datetime
from pprint import pprint

#To get HTML Text
def getHTMLText(url):
    kv = {'user-agent':'Mozilla/5.0'}
    try:
        r = requests.get(url, timeout=30, headers = kv)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.text
    except:
        return ""

In [58]:
wcaid = '2018shen04'
rootURL = 'https://www.worldcubeassociation.org/'
resultHTML = getHTMLText(rootURL + 'persons/' + wcaid)
soup = bs(resultHTML, 'html.parser')

In [59]:
soup

<!DOCTYPE html>

<html>
<head>
<script>window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"43326a088c","applicationID":"10385163","transactionName":"dFheEERaWFwHFE1CVUVEXwpFGkdYDRE=","queueTime":0,"applicationTime":75,"agent":""}</script>
<script>(window.NREUM||(NREUM={})).loader_config={licenseKey:"43326a088c",applicationID:"10385163"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var i=n[t]={exports:{}};e[t][0].call(i.exports,function(n){var i=e[t][1][n];return r(i||n)},i,i.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(e,n,t){function r(){}function i(e,n,t){return function(){return o(e,[u.now()].concat(f(arguments)),n?null:this,t),n?void 0:this}}var o=e("handle"),a=e(4),f=e(5),c=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setC

In [126]:
# 获取所有比赛名和对应日期的字典
comp_date_dic = {}
all_ = [item.strip().strip(',') for item in soup.find_all('script')[-1].get_text().strip().split('\n')]
i = 0
while i < len(all_):
    if all_[i][:6] == '"name"':
        comp = all_[i].split(':')[1].strip().strip('"')
        i += 4
        date = all_[i].split(':')[1].strip().strip('"')
        if '-' in date:
            date = date.split('-')
            date[0] = date[0].strip() + ', '
            date[1] = date[1][-4:]
            date = ''.join(date)
        date = ''.join(str(datetime.datetime.strptime(''.join(date), '%b %d, %Y')).split()[0].split('-'))
        comp_date_dic[comp] = date
    i += 1
pprint(comp_date_dic)

{'Hangzhou Open 2018': '20181110',
 'Hangzhou Open 2019': '20190518',
 'Linfen Open 2019': '20190707',
 'Ningbo Open 2018': '20181013',
 'SJTU Open 2018': '20180630',
 'SJTU Winter Open 2018': '20181230',
 'Shanghai Love in ECNU 2018': '20180519',
 'Shanghai Winter 2018': '20180120',
 'Suzhou Open 2019': '20190804',
 'Wuxi Open 2019': '20190810',
 'Wuxi Winter 2019': '20191208',
 "Xi'an Cherry Blossom 2018": '20180406'}


In [142]:
# 时间转换 eg. 1:20.73 -> 80.73
def format_time(t):
    if ':' in t:
        t = t.split(':')
        t = float(t[0]) * 60 + float(t[1])
    return float(t)

format_time('50.63')

50.63

In [159]:
# 获取在校期间最好成绩
entertime = '20170910'
leavetime = '20210701'

personal_best = {}
events = ['333', '222', '444', '555', '666', '777', '333bf', '333fm', '333oh', 'clock', 'minx', 'pyram', 'skewb', 'sq1', '444bf', '555bf', '333mbf']
for e in events:
    personal_best[e] = {'single':[float('inf'), '', '', ''], 'avg':[float('inf'), '', '', '']}
    
tbody_list = soup('tbody')
for tbody in tbody_list[3:]:
    event = tbody.get('class')[0].split('-')[-1]
    tr_list = tbody.find_all('tr', {'class':'result'})
    for tr in tr_list:
#         print(tr)
        temp = tr.find('td', {'class':'competition'}).text.strip()
        if temp:
            comp = temp
        date = comp_date_dic[comp]
        single = tr.find('td', {'class':'single'}).text.strip()
        single_time = format_time(single)
        try: 
            avg = tr.find('td', {'class':'average'}).text.strip()
            avg_time = format_time(avg)
        except: avg = avg_time = float('inf')
        print(event, comp, single_time, avg_time)
        if int(entertime) < int(date) < int(leavetime):
#         if True:
#             print(personal_best[event]['single'][0])
            if single_time < personal_best[event]['single'][0]:
                personal_best[event]['single'] = [single_time, single, comp, date]
            if avg_time < personal_best[event]['avg'][0]:
                personal_best[event]['avg'] = [avg_time, avg, comp, date]
#         break
#     break
    
    
pprint(personal_best)

333 Wuxi Winter 2019 12.36 14.12
333 Wuxi Winter 2019 13.94 14.97
333 Suzhou Open 2019 12.56 13.65
333 Suzhou Open 2019 12.22 13.49
333 Linfen Open 2019 12.35 15.23
333 Linfen Open 2019 12.8 14.1
333 Hangzhou Open 2019 12.97 14.79
333 Hangzhou Open 2019 13.03 14.49
333 SJTU Winter Open 2018 12.31 14.61
333 SJTU Winter Open 2018 12.76 15.32
333 Ningbo Open 2018 12.98 14.61
333 Ningbo Open 2018 13.46 15.09
333 SJTU Open 2018 14.21 15.37
333 SJTU Open 2018 11.4 16.46
333 Shanghai Love in ECNU 2018 14.02 16.46
333 Xi'an Cherry Blossom 2018 11.65 18.26
333 Shanghai Winter 2018 17.17 17.96
222 Linfen Open 2019 4.59 5.59
222 Hangzhou Open 2019 4.28 5.29
222 Hangzhou Open 2019 4.56 5.0
222 SJTU Open 2018 4.92 5.43
222 SJTU Open 2018 4.88 5.15
222 Xi'an Cherry Blossom 2018 4.17 5.71
444 Suzhou Open 2019 48.62 52.08
444 Hangzhou Open 2018 50.46 55.4
444 Ningbo Open 2018 47.19 48.59
444 Shanghai Winter 2018 57.88 62.42
555 Hangzhou Open 2018 80.73 92.25999999999999
555 Ningbo Open 2018 97.6 100.9